# RUN experiments

In this notebook, we will run all the required experiments using the `Experiment` class, which can be found at `sscode/experiment.py`, and can be seen below:

```python
    class Experiment(object):
        """
        This class Experiment summarizes all the previous work done with the linear and the
        knn models, as this class allows the user to perform a detailed analysis of one
        requested model given a set of parameters

        """

        def __init__(self, slp_data, wind_data, ss_data, # this must have several stations
                     sites_to_analyze: list = list(np.random.randint(10,1000,5)),
                     model: str = 'linear', # this is the model to analyze
                     model_metrics: list = [
                         'bias','si','rmse','pearson','spearman','rscore',
                         'mae', 'me', 'expl_var', # ...
                     ], # these are the metrics to evaluate
                     pca_attrs: dict = pca_attrs_default,
                     model_attrs: dict = linear_attrs_default):
            """
            As the initializator, the __init__ funciton creates the instance of the class,
            given a set of parameters, which are described below

            Args:
                slp_data (xarray.Dataset): These are the sea-level-pressure fields, previously
                    loaded with the Loader class, loader.predictor_slp!!
                wind_data (xarray.Dataset): These are the wind fields, previously
                    loaded with the Loader class, loader.predictor_wind!!
                ss_data (xarray.Dataset): This is the storm surge from the moana hindcast, previously
                    loaded with the Loader class, loader.predictand!!
                sites_to_analyze (list, optional): This is the list with all the moana v2
                    hindcast locations to analyze. Defaults to random locations.
                model (str, optional): Type of model to analyze. Defaults to 'linear'.
                model_metrics (list, optional): These are all the evaluation metrics that might
                    be used to evaluate the model performance. Defaults to simple list.
                pca_attrs (dict, optional): PCA dictionary with all the parameters to use.
                    Defaults to pca_attrs_default.
                model_attrs (dict, optional): Model dictionary with all the parameters to use. 
                    Defaults to linear_attrs_default.
            """

            # lets build the experiment!!
```

where it can be seen how, given the sea-level-pressure / wind data, the storm surge (it is preferable to use the Moana v2 hindcast nearshore), all the individual sites to analyze, the type of model that will be used (with its evaluation metrics), and the PCA attributes and the MODEL attributes that will be used to calculate the model.

Given these parameters, we construct the object of the class, and then running `execute_cross_model_calculations`, we can perform all the experiments given the possible combinations!!

```{warning}
Be careful with the ["curse of dimensionality"](https://en.wikipedia.org/wiki/Curse_of_dimensionality), as it seems very easy to add more options for the parameters of the models, but once we have more than 3-4 parameters with 2-3 different values, the number of models that will be performed can increase to hundreds!!
```

Below, different code cells with imports, loading functions, visualizations... will appear, but thw worflow is similar to all the notebooks in the repository. Once we load the data, we use the pre-built python classes to generate results!!

In [1]:
# basics
import os, sys
import progressbar

# arrays
import numpy as np
import pandas as pd
import xarray as xr

# plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# append sscode to path
sys.path.insert(0, os.path.join(os.path.abspath(''), '..'))

# custom
from sscode.config import data_path, default_location, \
    default_region, default_region_reduced
from sscode.data import Loader
from sscode.experiment import Experiment

# warnings
import warnings
warnings.filterwarnings('ignore')

# for autocomplete code
%config Completer.use_jedi = False

## Load the data, SLP and SS

Below, we load the slp fields and the Moana v2 hindcast data nearshore. Have three things in mind:

* First, the winds are **NOT** loaded, as we prefer to calculate the gradient of the slp fields. Having in mind that winds have more resolution (more computation is required) and the fact that we are playing around with projected winds, adds this new calculation to each step (and the Loader class is calculated outside the class Experiment, so the projedted winds are calculated just to one location).

* The `plot` parameter is set to `False`, but can be set to `True` if all the plots are wanted to be seen.

* The Moana v2 hindcast nearshore is used, and although the gridded netCDF data can be used, which includes ss data all over the New Zealand region, the data might be transformed to `dim=site` inside the dataset, as we did in previous notebooks.

In [2]:
# load the data
load_cfsr_moana_uhslc = Loader(
    data_to_load=['cfsr','moana','uhslc'], plot=False, 
    time_resample='1D', load_winds=True
)

# ---- maybe DAC reanalysis wants to be used ----
# dataset1 = load_cfsr_moana_uhslc.predictand
# dataset1_coords = ('longitude','latitude',None,'DAC ...')
# dataset1 = xr.Dataset(
#     {
#         'ss': (('time','site'), dataset1.values.reshape(
#             -1,len(dataset1[dataset1_coords[0]])*len(dataset1[dataset1_coords[1]]))),
#         dataset1_coords[0]: (('site'), list(dataset1[dataset1_coords[0]].values)*int(
#             (len(dataset1[dataset1_coords[0]])*len(dataset1[dataset1_coords[1]]))\
#                 /len(dataset1[dataset1_coords[0]]))),
#         dataset1_coords[1]: (('site'), np.repeat(dataset1[dataset1_coords[1]].values,
#             (len(dataset1[dataset1_coords[0]])*len(dataset1[dataset1_coords[1]]))\
#                 /len(dataset1[dataset1_coords[1]])))
#     }, coords={
#         'site': np.arange(len(dataset1[dataset1_coords[0]])*len(dataset1[dataset1_coords[1]])),
#         'time': dataset1.time.values
#     }
# ).dropna(dim='site',how='all')


 loading the sea-level-pressure fields... 


 loading daily resampled data... 


 loading the Moana v2 hindcast data... 


 loading and plotting the UHSLC tidal guages... 



## Create / Run -- Experiment object

In the two cells below, we first choose all the values that will have the different parameters for the model that will be used, and then we create the instance of the class `Experiment`, where different logs will appear before running all the models.

In [3]:
# experiment attributes
# ---------------------

sites_to_analyze = np.unique( # closest Moana v2 Hindcast to tidal gauges
    [ 689,328,393,1327,393,480,999,116,224,1124,949,708, # UHSLC
      1296,378,1124,780,613,488,1442,1217,578,200,1177,1025,689,949,224,1146, # LINZ
      1174,1260,1217,744,1064,1214,803,999 # OTHER (ports...)
    ]
)
pca_attrs_exp = {
    'calculate_gradient': [False,True],
    'winds': [False,True],
    'time_lapse': [1,2,3], # 1 equals to NO time delay 
    'time_resample': ['6H','12H','1D'], # 6H and 12H available...
    'region': [('local',(2.1,2.1)),('local',(4.1,4.1)),(True,default_region_reduced)]
}
linear_attrs_exp = {
    'train_size': [0.7], 'percentage_PCs': [0.98]
}
knn_attrs_exp = {
    'train_size': [0.7], 'percentage_PCs': [0.98],
    'k_neighbors': [None,3,6,9] # None calculates the optimum k-neighs
}

```{note}
Please check `Experiment` logs before running `execute_cross_model_calculations()`!!
```

In [4]:
# create the experiment
experiment = Experiment(
    load_cfsr_moana_uhslc.predictor_slp, load_cfsr_moana_uhslc.predictor_wind,
    # load_cfsr_moana_uhslc.predictor_wind,
    load_cfsr_moana_uhslc.predictand, # all the sites are passed to exp at first
    sites_to_analyze=sites_to_analyze, 
    model='linear', # model that will be used to predict
    model_metrics=['expl_var','mae','mse','me',
        'medae','tweedie', # check theory
        'ext_mae','ext_mse','ext_rmse','ext_pearson',
        'bias','si','rmse','pearson','spearman','rscore'
    ],
    pca_attrs=pca_attrs_exp,
    model_attrs=linear_attrs_exp
)


 The model has been correctly initialized with || model = linear ||             

 and model evaluation metrics = ['expl_var', 'mae', 'mse', 'me', 'medae', 'tweedie', 'ext_mae', 'ext_mse', 'ext_rmse', 'ext_pearson', 'bias', 'si', 'rmse', 'pearson', 'spearman', 'rscore']             

 pca_params = {'calculate_gradient': [False, True], 'winds': [False, True], 'time_lapse': [1, 2, 3], 'time_resample': ['6H', '12H', '1D'], 'region': [('local', (2.1, 2.1)), ('local', (4.1, 4.1)), (True, (160, 185, -30, -52))]} 

 model_params = {'train_size': [0.7], 'percentage_PCs': [0.98]}             

 which makes a total of 108 iterations as there are (2, 2, 3, 3, 3, 1, 1) values for each parameter             

 the experiment will be performed in sites = [ 116  200  224  328  378  393  480  488  578  613  689  708  744  780
  803  949  999 1025 1064 1124 1146 1174 1177 1214 1217 1260 1296 1327
 1442]             

 RUN CELL BELOW if this information is correct!!


```{note}
Once the instance of the class is well created, lets run all the models running the cell below
```

In [5]:
# finally, we can save the results, using the following function
def save_final_results(to_save_name: str = 
    '../data/statistics/experiments/experiment_linear_final.nc'):
    
    """
        This function saves the results in netCDF4 format
    """
    
    sites_datasets = [] # save all xarray.Dataset's in sites
    for site,exp_metrics_site in zip(experiment.ss_sites,exp_parameters):
        site_metrics = {}
        for im,metric in enumerate(experiment.model_metrics):
            site_metrics[metric] = (
                ('grad','winds','tlapse','tresample','region','tsize','perpcs'),
                exp_metrics_site[:,:,:,:,:,:,:,im]) 
        sites_datasets.append(xr.Dataset(site_metrics).expand_dims(
            {'site':[site]}
        ))
    experiment_metrics = xr.concat(sites_datasets,dim='site')
    experiment_metrics.to_netcdf(to_save_name)